In [ ]:
%load_ext autoreload
%autoreload 2

%store -r the_page
%store -r the_editor
%store -r agg_actions
%store -r editor_inputname
%store -r calculator


from IPython.display import display, Markdown as md
display(md("---"))
display(md(f"# A. Detecting conflicting editors"))
display(md(f"***Page: {the_page['title']}***"))
display(md(f"***Editor: {the_editor['name']}***"))

In [ ]:
conflicting_actions = calculator.actions[calculator.conflicts.shift(-1) & (
    calculator.actions.shift(-1)['editor']==editor_inputname)]

from IPython.display import display, Markdown as md
display(md('**List of conflicting editors:**'))
conflicting_editors = conflicting_actions['editor'].unique()
print('\n'.join(conflicting_editors))

display(md('**List of ids of conflicting editors:**'))
conflicting_editor_ids = set(int(x.split('|')[0]) for x in conflicting_editors)
print('\n'.join(str(x) for x in conflicting_editor_ids))

display(md('**List of conflicting revisions:**'))
conflicting_rev = conflicting_actions['rev_id'].unique()
print('\n'.join(str(x) for x in conflicting_rev))


In [ ]:
from IPython.display import display, Markdown as md
display(md("---"))
display(md(f"# B. Activity of conflicting editors"))
display(md(f"***Page: {the_page['title']}***"))
display(md(f"***Editor: {the_editor['name']}***"))

In [ ]:
conf_editor_agg_actions = agg_actions[agg_actions['editor_id'].isin(conflicting_editor_ids)]

# Listener
from visualization.actions_listener import ActionsListener
listener = ActionsListener(conf_editor_agg_actions)
actions = (conf_editor_agg_actions.loc[:,'total':'total_stopword_count'].columns.append(
    conf_editor_agg_actions.loc[:,'adds':'reins_stopword_count'].columns)).values.tolist()

# Visualization
from ipywidgets import interact
from utils.notebooks import get_date_slider_from_datetime
interact(listener.listen, 
         _range = get_date_slider_from_datetime(conf_editor_agg_actions['year_month']),
         granularity=['Monthly', 'Yearly'],
         black=actions, red=['total_surv_48h'] + actions + ['None'],
         green=['None'] + actions, blue=['None'] + actions)

In [ ]:
from IPython.display import display, Markdown as md
display(md("---"))
display(md(f"# B. Tokens of conflicting editors"))
display(md(f"***Page: {the_page['title']}***"))
display(md(f"***Editor: {the_editor['name']}***"))

In [ ]:
editor_conflicts = calculator.actions[calculator.actions['editor'].isin(conf_eds)]

#listener
from visualization.wordcloud_listener import WCListener
listener = WCListener(editor_conflicts)

# visualization
from utils.notebooks import get_date_slider_from_datetime
from ipywidgets import interact
interact(listener.listen, 
         _range=get_date_slider_from_datetime(editor_conflicts['rev_time']),
         action=['Both', 'Just Insertions', 'Just Deletions'],
         only_conflict = True)


In [ ]:
from IPython.display import HTML
from utils.notebooks import get_next_notebook

display(HTML(f'<a href="{get_next_notebook()}" target="_blank">Go to next workbook</a>'))
